In [1]:
####階層ベイズプロビットモデル####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [3]:
####データの発生####
#np.random.seed(8742)   #シードを設定

##データの設定
hh = 2000   #ユーザー数
k = 7   #パラメータ数
n = poisson(numpy.random.gamma(40, 1/1.5, hh), hh)   #1人あたりの観測数
N = np.sum(n)

In [4]:
##IDの設定
u_list = [i for i in range(hh)]
t_list = [i for i in range(hh)]
u_id = np.zeros(N, dtype='int')
t_id = np.zeros(N, dtype='int')
start = 0

for i in range(hh):
    #リストに格納
    u_list[i] = np.repeat(i, n[i])
    t_list[i] = np.array(range(n[i]))+1
    
    #リストをベクトル変換
    u_id[start:start+n[i]] = u_list[i]
    t_id[start:start+n[i]] = t_list[i]
    start += n[i] 

In [5]:
##説明変数の発生
intercept = np.transpose(np.matlib.repmat(1, 1, N))
Z1 = np.log(numpy.random.gamma(5, 1, N)) * numpy.random.binomial(1, 0.25, N)   #降水量の対数
Z2 = numpy.random.uniform(0, 1, N)   #チラシ商品の平均値引率
Z3 = np.log(numpy.random.poisson(numpy.random.gamma(25, 1/0.5, N)))   #チラシ掲載商品数の対数
Z3 = Z3 - np.min(Z3)
Z4 = numpy.random.binomial(1, 0.5, N)   #他店チラシ状況
Z = np.concatenate((intercept, Z1[:, np.newaxis], Z2[:, np.newaxis], Z3[:, np.newaxis], Z4[:, np.newaxis]), axis=1)   #データの結合

In [6]:
##パラメータの設定
beta0 = numpy.random.normal(-0.4, 0.4, hh)
beta1 = numpy.random.normal(-0.5, 0.3, hh)
beta2 = numpy.random.normal(0.5, 0.25, hh)
beta3 = numpy.random.normal(0.3, 0.25, hh)
beta4 = numpy.random.normal(-0.6, 0.2, hh)
beta = betat = np.concatenate((beta0[:, np.newaxis], beta1[:, np.newaxis], beta2[:, np.newaxis], beta3[:, np.newaxis],
                       beta4[:, np.newaxis]), axis=1)   #回帰係数の結合

In [7]:
##応答変数の発生
mu = np.sum(Z * beta[u_id, :], axis=1)   #潜在効用の平均構造
U = numpy.random.normal(mu, 1, N)   #潜在効用を生成
y = (U >= 0)*1 + (U < 0)*0   #購買有無に変換
np.mean(y)

0.4382087652981025

In [44]:
####マルコフ連鎖モンテカルロ法で階層ベイズプロビットモデルを推定####
##切断正規分布の乱数を発生させる関数
def rtnorm(mu, sigma, a, b, n):
    FA = norm.cdf(a, mu, sigma)
    FB = norm.cdf(b, mu, sigma)
    return norm.ppf(uniform(0, 1, n)*(FB-FA)+FA, mu, sigma)

In [49]:
##アルゴリズムの設定
R = 5000
keep = 4
burnin = int(1000/keep)
disp = 8
iter = 0
k = Z.shape[1]

##事前分布の設定
#プロビットモデルの事前分布
sigma = 1
beta0 = 0
tau0 = 1/100

#階層モデルの事前分布
Bbar = np.repeat(0, k)
A = 0.01 * np.identity(k)
nu = k*2
V = nu * np.identity(k)

In [50]:
##初期値の設定
#回帰係数の初期値
beta_mu = np.dot(np.linalg.inv(np.dot(Z.T, Z)), np.dot(Z.T, y))
beta = np.random.multivariate_normal(beta_mu, 0.1 * np.identity(k), hh)

#階層モデルの初期値
alpha = beta_mu
Cov = 0.1 * np.identity(k)
Cov_inv = np.linalg.inv(Cov)

In [51]:
##パラメータ格納用配列
BETA = np.zeros((hh, k, int(R/keep)))
ALPHA = np.zeros((int(R/keep), k))
COV = np.zeros((k, k, int(R/keep)))

##インデックスを作成
index = np.array(range(N))
index_u = [i for i in range(hh)]

for i in range(hh):
    index_u[i] = index[u_id==i]

##切断領域を定義
a = (1-y)*(-100) + y*0
b = y*100 + (1-y)*0

##定数を設定
XX = [i for i in range(hh)]
XX_inv = [i for i in range(hh)]
for i in range(hh):
    index = index_u[i]
    XX[i] = np.dot(Z[index, :].T, Z[index, :])

In [52]:
####ギブスサンプリングでパラメータをサンプリング####
for rp in range(R):
    ##切断正規分布から潜在効用を生成
    mu = np.sum(Z * beta[u_id, :], axis=1)   #潜在効用の平均構造
    U = rtnorm(mu, sigma, a, b, N)   #潜在効用を生成

    ##個人別にbetaをサンプリング
    for i in range(hh):
        #データを抽出
        index = index_u[i]
        Zi = Z[index, :]

        #回帰係数の事後分布のパラメータ
        Xy = np.dot(Zi.T, U[index]) 
        XXV = np.linalg.inv(XX[i] + Cov_inv)
        beta_mu = np.dot(XXV, Xy + np.dot(Cov_inv, alpha))

        #多変量正規分布からbetaをサンプリング
        beta[i, :] = np.random.multivariate_normal(beta_mu, sigma*XXV, 1)

    ##階層モデルのパラメータをサンプリング
    #多変量正規分布から階層モデルの回帰係数をサンプリング
    alpha_mu = hh/(hh+tau0) * np.mean(beta, axis=0)
    alpha = np.random.multivariate_normal(alpha_mu, Cov/(hh + tau0))

    #逆ウィシャート分布から階層モデルの分散共分散行列をサンプリング
    V_par = V + np.dot(beta.T, beta)
    Sn = nu + hh
    IW = ss.invwishart(df=Sn, scale=V_par)
    Cov = IW.rvs(1)
    Cov_inv = np.linalg.inv(Cov)

    ##サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        BETA[:, :, mkeep] = beta
        ALPHA[mkeep, :] = alpha
        COV[:, :, mkeep] = Cov

    if rp%disp==0:
        pr = norm.cdf(mu, 0, sigma)
        LL = sum(np.log(scipy.stats.binom.pmf(y, 1, pr)))
        print(rp)
        print(LL)
        print(alpha)

0
-48724.99141335901
[ 0.26022348 -0.23274155  0.13736342 -0.00129787 -0.29303266]
8
-29398.461889609047
[ 0.18802508 -0.53575602  0.41926882 -0.06091124 -0.64989359]
16
-29158.228642628375
[ 0.03729311 -0.6181063   0.50239191  0.01513336 -0.65844707]
24
-29076.923566983045
[-0.11662445 -0.58635882  0.51391178  0.115278   -0.63813067]
32
-28959.11171860189
[-0.23847567 -0.61930395  0.53006449  0.19067206 -0.67401809]
40
-29025.950104599942
[-0.31538441 -0.63864514  0.5233124   0.24811979 -0.66589179]
48
-28903.681760292395
[-0.35097174 -0.62768815  0.56224534  0.25845941 -0.62967753]
56
-28962.84440150316
[-0.51258643 -0.6346139   0.5647484   0.37826    -0.65674433]
64
-28831.873233758295
[-0.49159147 -0.62006976  0.57517488  0.37889879 -0.66409511]
72
-28864.40588925927
[-0.5130251  -0.64955497  0.55901651  0.39159124 -0.65260077]
80
-28848.054073016625
[-0.47747141 -0.63191114  0.55441422  0.36260846 -0.65368017]
88
-28827.608727949362
[-0.5651355  -0.68420041  0.60816385  0.42333705

KeyboardInterrupt: 

In [53]:
np.mean(BETA, axis=2)

array([[-0.64485683, -0.72497735,  0.63975048,  0.69848774, -0.61361864],
       [-0.47927302,  0.15606027,  0.04870082, -0.08319408,  0.08079607],
       [-0.5455734 , -0.51300754,  0.82732481,  0.07612754, -0.81104166],
       ...,
       [-0.19571088, -0.46514351,  0.34607423, -0.06452344, -0.60764   ],
       [-0.02439096, -0.24406381,  0.10865465,  0.27102787, -0.25775044],
       [-0.14429498, -0.23462586,  0.24092543,  0.3168321 , -0.27686554]])

In [356]:
np.random.multivariate_normal(alpha_mu, Cov/(hh + tau0), 1)

array([[ 0.25634946, -0.22395765,  0.16446298, -0.00753195, -0.29143252]])

In [282]:
i = 3
print(y[i])
print(norm.cdf(mu[i], 0, sigma))
print(scipy.stats.binom.pmf(y[i], 1, norm.cdf(mu[i], 0, sigma)))

0
0.315393440713
0.684606559287
